# Kolla nya idéer och features 
### Speciellt för att prioritera mellan hästar i olika lopp
- kelly vs proba
- antal hästar i loppen som feature
- motståndarinfo
  - De tre bäst rankade motståndarna i loppet, deras antal streck
  - genomsnittet av någon ranking för hästarna
  - ...




## 1. En sorterad lista med proba och/eller kelly 
Från den listan plockas hästar tills max insats är nådd, kanske lltid spika en eller två hästar

In [ ]:
# moduler
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
import flaml
import pickle
from IPython.display import display
# # read a model from a file with pickle
with open('..\\modeller\\FLAML_model.sav', 'rb') as f:
    model = pickle.load(f)
with open('..\\modeller\\FLAML2_model.sav', 'rb') as f:
    model2 = pickle.load(f)
    
pd.set_option('display.width', 100)


In [ ]:
def remove_features(df_, remove_mer=[]):
    df = df_.copy()
    df.drop([ 'startnr', 'avd','vodds', 'podds', 'bins', 'h1_dat',
              'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1, inplace=True)
    if remove_mer:
            df.drop(remove_mer, axis=1, inplace=True)

    return df


def load_enc():
    with open('..\\modeller/encoder.sav', 'rb') as file:
        enc = pickle.load(file)
    return enc
def ordinal_enc(df_, feature):
    enc = load_enc()
    df = df_.copy()
    enc.transform(df[[feature]])
    # enc.fit_transform(df[[feature]])
    df[[feature]] = enc.transform(df[[feature]])
    return df

In [ ]:
# skapa data
df = pd.read_csv('..\\all_data.csv')
X=df.copy()
X.drop('plac', axis=1, inplace=True)
X = ordinal_enc(X, 'häst')
y=(df.plac==1)*1   # plac 1 eller 0


In [ ]:
# model1 är FLAML och model2 är FLAML2
proba = model.predict_proba(remove_features(X))
proba2 = model2.predict_proba(remove_features(X))
X['proba'] = proba[:,1]
X['proba2'] = proba2[:, 1]


In [ ]:
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    if odds is None:
        o = 100/streck.copy()
    else:
        o=odds.copy()
            
    # for each values > 50 in odds set to 1
    o[o>40]=1
    return (o*proba - (1-proba))/o
            

Xtst = pd.read_csv('..\\sparad_scrape.csv')

#df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
Xtst['kellys'] = kelly(Xtst.proba, Xtst.streck+1, None)
proba2 = model2.predict_proba(remove_features(Xtst))
Xtst['proba2'] = proba2[:, 1]
Xtst['kelly2'] = kelly(Xtst.proba2, Xtst.streck+1, None)


In [ ]:
Xtst[['avd' , 'startnr', 'proba','kellys','proba2','kelly2']]

## plocka hästar efter proba och kelly till en df
1. Upp till 300:-
2. Minst en häst per avd
3. Lopp utan favoriter får utspridda låga värden och väljs inte för "helgardering" - hur lösa detta?
   

In [ ]:
def compute_total_insats(df):
    insats=0
    # group by avd
    summa = df.groupby('avd').avd.count().prod() / 2   
    return summa

def fill_up_dfSpel(df_, max_insats=300,metod='proba'):
    # dfSpel är de hästar som ska spela
    df=df_.copy()
    dfSpel = pd.DataFrame()
    df=df.sort_values(by=metod, ascending=[False])
    curr_insats = 0
    for index, row in df.iterrows():
        dfSpel = dfSpel.append(row)
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats>max_insats+30:
            dfSpel = dfSpel.iloc[:-1,:]
            break
        # print(curr_insats)

    print(compute_total_insats(dfSpel))
    return dfSpel


## Lägg in antal hästar som feature

In [ ]:
def lägg_in_antal_hästar(df_):
    df = df_.copy()
    # count number per avdeling
    # df_count = df.groupby(['datum', 'avd'])['avd'].count()
    df['ant_per_lopp'] = None
    df['ant_per_lopp'] = df.groupby(['datum', 'avd'])['avd'].transform('count')

    # drop startnr because it has too many NaN
    # drop vodds, podds, bins because they are not known before v75
    X = df.drop(['plac', 'startnr', 'vodds', 'podds', 'bins',   # behåll avd som tas bort senare
                'h1_dat', 'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat', ], axis=1)

    y = (df['plac'] == 1)*1

    return X, y
X,y = lägg_in_antal_hästar(df)

In [ ]:
X.columns

In [ ]:
# använda catboostClassifier
from catboost import CatBoostClassifier, Pool,cv


In [ ]:
# get numerical features and cat_features
num_features = list(X.drop('avd', axis=1).select_dtypes(include=[np.number]).columns)
cat_features = list(X.drop('avd', axis=1).select_dtypes(include=['object']).columns)
print(f'Rätt längd på summan (med datum)? {1+len(num_features)+len(cat_features)} == {len(X.columns)}')
# check cat_features isna
print('NaN before:', X[cat_features].isna().sum()[
      X[cat_features].isna().sum() > 0].sort_values(ascending=False).sum())

# impute 'missing' for all NaN in cat_features
X[cat_features] = X[cat_features].fillna('missing')
print('NaN after:', X[cat_features].isna().sum().sum())


In [ ]:
"""
cross validation with catboost
"""
def catboost_cv(X, y, cat_features, fold_count=5):

    # Create a pool
    pool = Pool(data=X, label=y, cat_features=cat_features)
    params = {"iterations": 1000,
              "loss_function": "Logloss",
              "verbose": False}

    cv_results = cv(pool, params, fold_count=fold_count,

                    early_stopping_rounds=100,
                    # loss_function='Logloss',
                    shuffle=False,
                    type='TimeSeries',
                    verbose=False)
    return cv_results


In [ ]:
result = catboost_cv(X.drop('avd', axis=1), y, cat_features)

In [ ]:
result['test-Logloss-mean'].min()


med  antal hästar som feature: 0.2396187999030749  
utan antal hästar som feature: 0.23964830518637106  
Minimalt bättre men det är inte vad jag vill uppnå utan bättre viktning mellan hästar i olika lopp

# Tester med olika lösningar 

In [1]:
# moduler
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from IPython.display import display
pd.set_option('display.width', 100)


In [2]:
# skapa data
df = pd.read_csv('..\\all_data.csv')
X = df.copy()
X.drop('plac', axis=1, inplace=True)
# X = ordinal_enc(X, 'häst')
y = (df.plac == 1)*1   # plac 1 eller 0


def get_data_for_catboost(X):
    # get numerical features and cat_features
    num_features = list(X.drop('avd', axis=1).select_dtypes(
        include=[np.number]).columns)
    cat_features = list(X.drop('avd', axis=1).select_dtypes(
        include=['object']).columns)
    print(f'Rätt längd på summan (med datum)? {1+len(num_features)+len(cat_features)} == {len(X.columns)}')
    # check cat_features isna
    print('NaN before:', X[cat_features].isna().sum()[X[cat_features].isna().sum() > 0].sort_values(ascending=False).sum())

    # impute 'missing' for all NaN in cat_features
    X[cat_features] = X[cat_features].fillna('missing')
    print('NaN after:', X[cat_features].isna().sum().sum())
    return X, cat_features

X, cat_features = get_data_for_catboost(X)
# cat_features = list(X.drop('avd', axis=1).select_dtypes(include=['object']).columns)
print('cat_features:',cat_features)

# antal hästar per avdeling
def lägg_in_antal_hästar(df_):
    df = df_.copy()
    df['ant_per_lopp'] = None
    df['ant_per_lopp'] = df.groupby(['datum', 'avd'])['avd'].transform('count')
    return df

# mest streck per avdeling
def mest_streck(X_, i, datum, avd):
    X = X_.copy()
    X.sort_values(by=['datum', 'avd', 'streck'], ascending=[
                  True, True, False], inplace=True)
    return X.loc[(X.datum == datum) & (X.avd == avd), 'streck'].iloc[i]

# mest streck per avd som fetures (n bästa)
def lägg_in_motståndare(X_, ant_motståndare):
    X = X_.copy()

    # set X['motståndare1'] to largest streck in every avd
    grouped = X.groupby(['datum', 'avd'])['streck']
    X['motståndare1'] = grouped.transform(max)

    for i in range(2, ant_motståndare+1):
        # set X['motståndare'+str(i)] to ith largest streck in every avd
        X['motståndare' + str(i)] = grouped.transform(lambda x: x.nlargest(i).min())

    return X

# diff streck per avd som fetures (n största diffarna)
def lägg_in_diff_motståndare(X_, motståndare):
    X = X_.copy()

    # set X['motståndare1'] to largest streck in every avd
    grouped = X.groupby(['datum', 'avd'])['streck']
    X['diff1'] = grouped.transform(max) - X.streck

    for i in range(2, motståndare+1):
        # set X['motståndare'+str(i)] to ith largest streck in every avd
        X['diff' + str(i)] = grouped.transform(lambda x: x.nlargest(i).min()) - X.streck

    return X

# Skapa X_train och X_test
def create_X_train_test(X, y, datum):
    X_train = X[X.datum < datum]
    X_test = X[X.datum == datum]
    y_train = y[X.datum < datum]
    y_test = y[X.datum == datum]
    return X_train, X_test, y_train, y_test

#make häst, bana, kusk lower case

for f in ['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
    # print(X[f], type(X[f]))
    X[f] = X[f].str.lower()


Rätt längd på summan (med datum)? 78 == 78
NaN before: 369
NaN after: 0
cat_features: ['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_dat', 'h1_kusk', 'h1_bana', 'h2_dat', 'h2_kusk', 'h2_bana', 'h3_dat', 'h3_kusk', 'h3_bana', 'h4_dat', 'h4_kusk', 'h4_bana', 'h5_dat', 'h5_kusk', 'h5_bana']


In [3]:
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    if odds is None:
        o = 100/streck.copy()
    else:
        o = odds.copy()

    # for each values > 50 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o

In [8]:
# pick the häst to play
def compute_total_insats(df):
    insats = 0
    # group by avd
    summa = df.groupby('avd').avd.count().prod() / 2
    return summa

# för en omgång (ett datum) ta ut största diff för streck per avd men bara om diff >= 25
def största_diff_per_avd(df_, ant,only_clear):
    df=df_.copy()
    min_diff = 25 if only_clear else 0
    # sortera på avd,streck
    df = df.sort_values(['avd','streck'], ascending=[False, False])
    diff_list = []
    for avd in range(1,8):
        diff = df.loc[df.avd==avd].streck.iloc[0] - df.loc[df.avd==avd].streck.iloc[1]
        if diff >= min_diff:
            diff_list.append((avd,diff))

     # sortera på diff
    diff_list = sorted(diff_list, key=lambda x: x[1], reverse=True)   
    return diff_list[:ant]

# temp is a list of tuples (avd, diff). check if avd is in the list
def check_avd(avd, temp):
    for t in temp:
        if t[0] == avd:
            return True
    return False   


In [94]:
  
def spela(df_, max_insats=300, proba=True, kelly=True,ant_favoriter=0,streck=True,only_clear=False):
    df = df_.copy()
    # print('omg df',df.shape)
    dfSpel = pd.DataFrame()
    if proba:
        df = df.sort_values(by='proba', ascending=False)
        if kelly:
            df2 = df.sort_values(by='kelly', ascending=False)
    else:
        df = df.sort_values(by='kelly', ascending=False) # must be kelly
        
    # se till att vi har för alla avd. Välj den bästa per avd
    if proba:
        temp = df.sort_values(by=['proba'], ascending=False)
    else:
        temp = df.sort_values(by=['kelly'], ascending=False)    
    for avd in range(1, 8):
        dfSpel = dfSpel.append(temp[temp.avd == avd].iloc[0])
    #
    favorit_list = största_diff_per_avd(df, ant_favoriter,only_clear)
    curr_insats = 0
    cnt=0
    for index, row in df.iterrows():
        if check_avd(row.avd, favorit_list): # Inga fler hästar för avd med en favorit
            continue
        
        dfSpel = dfSpel.append(row)
        # print(dfSpel.shape)
        # print('row',row.shape)
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats > max_insats+30:
            dfSpel = dfSpel.iloc[:-1, :]
            break
        if kelly & proba:
            row2 = df2.iloc[cnt]
            # print('row2',row2.shape)
            dfSpel = dfSpel.append(row2)
            # remove duplicates in dfSpel nd keep the first
        dfSpel = dfSpel.drop_duplicates(subset=['avd', 'häst'], keep='first')
        # print('dfSpel',dfSpel.shape)    
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats > max_insats+30:
            dfSpel = dfSpel.iloc[:-1, :]
            break
        cnt +=1
                
    return dfSpel

# train with curr_datum
def learn(X_train, y_train, X_test, y_test, cat_features):
    train_pool = Pool(X_train.drop('avd', axis=1), label=y_train, cat_features=cat_features)
    test_pool = Pool(X_test.drop('avd', axis=1), label=y_test, cat_features=cat_features)
    cbc = CatBoostClassifier(iterations=1000, loss_function='Logloss', eval_metric='AUC', verbose=False)
    cbc.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100,use_best_model=True, verbose=False)
    return cbc

def beräkna_utdelning(dfSpel,ant_rätt, df_utdelning):
    if ant_rätt==7:
        slista = list(dfSpel.groupby('avd').avd.count()-1)
        sjuor = 1
        sexor = sum(slista)
        femmor =0
        for avd_ix in range(0, 7):
            for avd2_ix in range(avd_ix+1, 7):
                femmor += slista[avd_ix] * slista[avd2_ix]
    
    elif ant_rätt==6:
        sjuor=0
        avd_list = list(dfSpel.loc[dfSpel.rätt == 'OK']['avd'])  # avd med rätt
        missing = [x for x in range(1, 8) if x not in avd_list]
        sexor = dfSpel.loc[dfSpel.avd == missing[0], 'avd'].count()
        femmor=0
        for avd in avd_list:
            femmor+=dfSpel.loc[dfSpel.avd==avd, 'avd'].count()
        femmor=(femmor-6) * sexor
        
    elif ant_rätt==5:
        sjuor,sexor=0,0
        avd_list = list(dfSpel.loc[dfSpel.rätt == 'OK']['avd'])  # avd med rätt
        missing = [x for x in range(1, 8) if x not in avd_list]
        femmor=dfSpel.loc[dfSpel.avd==missing[0], 'avd'].count() + dfSpel.loc[dfSpel.avd==missing[1], 'avd'].count()
    else:
        sjuor,sexor,femmor = 0,0,0 
           
    datum = dfSpel.iloc[0].datum       
    priser = list(df_utdelning.loc[df_utdelning.datum ==datum, ['7rätt', '6rätt', '5rätt']].values[0])
    return sjuor,sexor,femmor, priser[0]*sjuor+ priser[1]*sexor + priser[2]*femmor

def rätta(y_validate, dfSpel, df_utdelning):
    yindex = y_validate.loc[y == 1].index
    # print(1,_,dfSpel.shape)
    ant_rätt = len(dfSpel.loc[dfSpel.index.isin(yindex)])
    # print(f': total insats {compute_total_insats(dfSpel)} med {dfSpel.shape[0]} spelade hästar: {ant_rätt} rätt')
    dfSpel['rätt'] = ' '
    dfSpel[['avd', 'streck','rätt']]
    dfSpel.loc[dfSpel.index.isin(yindex), 'rätt'] = 'OK'
    # print(2,_,dfSpel.shape)
    return  beräkna_utdelning(dfSpel, ant_rätt, df_utdelning)


## Loop över datum, learn+predict

In [120]:

def testa_flera_omg(X_,y_,typ, max_cnt=60): 
    X = X_.copy()
    y = y_.copy()

    # skriv ut parametrar som styr vilken lösning som testas
    max_insats = stats_typer.loc[stats_typer.typ == typ].insats.values[0]
    print('max_insats',max_insats)
    bProba = stats_typer.loc[stats_typer.typ == typ].proba.values[0]
    print('Proba',bProba)
    bKelly = stats_typer.loc[stats_typer.typ == typ].kelly.values[0]
    print('Kelly',bKelly)
    bAnt_hästar = stats_typer.loc[stats_typer.typ == typ].ant_hästar.values[0]
    ant_favoriter = stats_typer.loc[stats_typer.typ == typ].favoriter.values[0]
    only_clear = stats_typer.loc[stats_typer.typ == typ].only_clear.values[0]
    print('ant_favoriter',ant_favoriter, 'only_clear =',only_clear)
    motståndare = stats_typer.loc[stats_typer.typ == typ].motst_info.values[0]
    diff = stats_typer.loc[stats_typer.typ == typ].motst_diff.values[0]
    streck = stats_typer.loc[stats_typer.typ == typ].streck.values[0]
    
    if diff:
        print('Diff motståndare',motståndare)
    else:
        print(motståndare, ' motståndare')
        
    if bAnt_hästar:
        X = lägg_in_antal_hästar(X)
        print('lägg_in_antal_hästar per avd')
     
    if not streck:
        print('not streck')
          
    df_utdelning = pd.read_csv('..\\utdelning.csv')
    datum = df.datum.unique()
    spel_dict = {'typ':[],'datum':[],'insats':[], 'utdelning':[], 'sjuor':[], 'sexor':[], 'femmor':[]}
    kassa=0
    ix = len(datum)//2+100
    if motståndare>0:
        if diff:
            X=lägg_in_diff_motståndare(X, motståndare)
        else:    
            X=lägg_in_motståndare(X, motståndare)
        # print(X)
        
    for cnt,curr_datum in enumerate(datum[ix:-1]):
        # select data
        X_train = X.loc[df.datum<curr_datum].copy()
        y_train = y.loc[df.datum<curr_datum].copy()
        X_test = X.loc[df.datum>curr_datum].copy()
        y_test = y.loc[df.datum>curr_datum].copy()
        X_validate = X.loc[df.datum==curr_datum].copy()
        y_validate = y.loc[df.datum==curr_datum].copy()
        if not streck:
            X_train.drop('streck', axis=1, inplace=True)
            X_test.drop('streck', axis=1, inplace=True)
            # keep in X_validate for kelly formula
        
        # Learn
        cbc = learn(X_train, y_train, X_test, y_test, cat_features)
        
        # predict
        if not streck:
            validate_pool = Pool(X_validate.drop(['avd', 'streck'], axis=1),label=y_validate, cat_features=cat_features)
        else:
            validate_pool = Pool(X_validate.drop(['avd'], axis=1), label=y_validate, cat_features=cat_features)
        
        X_validate['proba'] = cbc.predict_proba(validate_pool)[:, 1]
        X_validate['kelly'] = kelly(X_validate.proba, X_validate.streck+1, None)
        
        # Spela
        dfSpel = spela(X_validate, max_insats=max_insats, proba=bProba, kelly=bKelly, 
                       ant_favoriter=ant_favoriter,streck=streck,only_clear=only_clear)
        
        # Rätta
        sjuor, sexor, femmor, utdelning = rätta(y_validate, dfSpel, df_utdelning)
        insats = compute_total_insats(dfSpel) 
        ant_spelade = dfSpel.shape[0]
        kassa += utdelning-insats
        print(f'{curr_datum} - insats: {insats}, ant spel: {ant_spelade}, rätt=({sjuor},{sexor},{femmor}), Utd: {utdelning}  :\tkassa: {kassa}')
        # append insats, utdelning, sjuor to spel_dict['datum']
        spel_dict['typ'].append(typ)
        spel_dict['datum'].append(curr_datum)
        spel_dict['insats'].append(insats)
        spel_dict['utdelning'].append(utdelning)
        spel_dict['sjuor'].append(sjuor)
        spel_dict['sexor'].append(sexor)
        spel_dict['femmor'].append(femmor)
        
        if cnt+1>=max_cnt:
            break


    print('Finished')
    return spel_dict,dfSpel


stats_typer = pd.DataFrame(columns=['typ', 'ant_datum', 'insats', 'ant_hästar',
                           'proba', 'kelly', 'motst_diff', 'motst_info', 'favoriter','only_clear', 'streck'])
stats_typer.loc[len(stats_typer)] =  ['1',  200, 300, True,  True,  True, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['2',  200, 300, True,  True, False, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['3',  200, 300, True, False,  True, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['4',  200, 300, False, True,  True, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['5',  200, 300, False, True, False, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['6',  200, 300, False,False,  True, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['7',  200, 3000, True, True,  True, False, 0, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['8',  200, 300, True,  True,  True, False, 0, 2, False, True]
stats_typer.loc[len(stats_typer)] =  ['9',  200, 300, True,  True,  True, False, 3, 0, False, True]
stats_typer.loc[len(stats_typer)] =  ['10', 200, 300, True,  True,  True, False, 3, 2, False, True]
stats_typer.loc[len(stats_typer)]  = ['11', 200, 300, True,  True,  True, False, 1, 2, False, True]
stats_typer.loc[len(stats_typer)]  = ['12', 200, 300, True,  True,  True, False, 1, 1, False, True]
stats_typer.loc[len(stats_typer)]  = ['13', 200, 300, True,  True,  True, False, 2, 2, False, True]
stats_typer.loc[len(stats_typer)]  = ['14', 200, 300, True,  True,  True, True,  2, 2, False, True]
stats_typer.loc[len(stats_typer)]  = ['15', 200, 300, True,  True,  True, True,  3, 2, False, True]
stats_typer.loc[len(stats_typer)]  = ['16', 200, 300, True,  True,  True, False, 2, 2,  True, True]
stats_typer.loc[len(stats_typer)]  = ['17', 200, 300, True,  True,  True, True,  2, 2,  True, True]
stats_typer.loc[len(stats_typer)]  = ['18', 200, 300, True,  True,  True, True,  3, 2,  True, True]
stats_typer.loc[len(stats_typer)]  = ['19', 200, 300, True,  True,  True, False, 0, 0, False, False]
stats_typer.loc[len(stats_typer)]  = ['20', 200, 300, True,  True,  True, True,  2, 2,  True, False]
stats_typer.loc[len(stats_typer)] =  ['21', 200, 300, False, True, False, True,  2, 2,  True, False]
stats_typer

### Kör tidigare datum
### Använd diff till motståndare
### Plocka endast favoriter som är solklara
### Hantera avd utan favoriter med många jämna


,typ,ant_datum,insats,ant_hästar,proba,kelly,motst_diff,motst_info,favoriter,only_clear,streck
0,1,200,300,True,True,True,False,0,0,False,True
1,2,200,300,True,True,False,False,0,0,False,True
2,3,200,300,True,False,True,False,0,0,False,True
3,4,200,300,False,True,True,False,0,0,False,True
4,5,200,300,False,True,False,False,0,0,False,True
5,6,200,300,False,False,True,False,0,0,False,True
6,7,200,3000,True,True,True,False,0,0,False,True
7,8,200,300,True,True,True,False,0,2,False,True
8,9,200,300,True,True,True,False,3,0,False,True
9,10,200,300,True,True,True,False,3,2,False,True


## Kör tester

In [121]:
curr_typ = '21'
spel_dict, dfSpel = testa_flera_omg(X,y, curr_typ, 200)
temp_stats = pd.DataFrame(spel_dict)
temp_stats.typ=temp_stats.typ.astype('string')

max_insats 300
Proba True
Kelly False
ant_favoriter 2 only_clear = True
Diff motståndare 2
not streck
2019-08-21 - insats: 320.0, ant spel: 21, rätt=(0,0,0), Utd: 0  :	kassa: -320.0
2019-08-22 - insats: 288.0, ant spel: 20, rätt=(0,0,0), Utd: 0  :	kassa: -608.0
2019-08-23 - insats: 240.0, ant spel: 19, rätt=(0,0,0), Utd: 0  :	kassa: -848.0
2019-08-24 - insats: 288.0, ant spel: 20, rätt=(0,0,5), Utd: 1035  :	kassa: -101.0
2019-08-25 - insats: 225.0, ant spel: 20, rätt=(0,5,45), Utd: 210  :	kassa: -116.0
2019-08-31 - insats: 240.0, ant spel: 20, rätt=(0,0,5), Utd: 105  :	kassa: -251.0
2019-09-01 - insats: 270.0, ant spel: 20, rätt=(0,0,0), Utd: 0  :	kassa: -521.0
2019-09-07 - insats: 288.0, ant spel: 19, rätt=(0,0,4), Utd: 88  :	kassa: -721.0
2019-09-08 - insats: 288.0, ant spel: 20, rätt=(0,0,0), Utd: 0  :	kassa: -1009.0
2019-09-14 - insats: 240.0, ant spel: 19, rätt=(0,1,12), Utd: 398  :	kassa: -851.0
2019-09-21 - insats: 270.0, ant spel: 20, rätt=(0,0,0), Utd: 0  :	kassa: -1121.0
2019

In [122]:
temp_stats.loc[temp_stats.utdelning > 0][['datum','utdelning', 'sjuor', 'sexor', 'femmor']].sort_values(by='utdelning', ascending=False).head(60)


,datum,utdelning,sjuor,sexor,femmor
148,2021-09-25,7496,1,14,75
83,2020-08-29,7293,1,14,75
149,2021-10-09,3747,1,13,67
166,2022-01-22,1840,0,2,20
100,2020-11-14,1584,1,15,82
140,2021-08-14,1561,1,14,74
62,2020-05-31,1467,1,14,75
136,2021-07-10,1381,1,13,66
128,2021-05-22,1300,0,2,22
21,2019-11-16,1238,1,13,64


In [123]:
spel_stats = pd.read_csv('spel_stats.csv')
spel_stats = pd.concat([spel_stats, temp_stats], ignore_index=True)
spel_stats.typ = spel_stats.typ.astype('string')
spel_stats.drop_duplicates(subset=['typ', 'datum'], keep='last', inplace=True)
spel_stats.to_csv('spel_stats.csv', index=False)


In [124]:
def stats_for_typ(typ,stats):
    stats_row = stats.loc[stats.typ == typ]
    print(stats_row[['insats','utdelning','sjuor','sexor','femmor']].sum())
    print(f'Kassa (typ={typ}):',stats_row.utdelning.sum()-stats_row.insats.sum())
stats_for_typ(curr_typ, spel_stats)

def bästa_kassa(stats):
    typlista=[]
    for typ in stats.typ.unique():
        row = stats.loc[stats.typ==typ]
        kassa=row.utdelning.sum()-row.insats.sum()
        typlista.append((typ,kassa))
    # sort a list descending
    typlista.sort(key=lambda x: x[1], reverse=True)
    
    print(typlista)
bästa_kassa(spel_stats)


insats       48562.0
utdelning    44500.0
sjuor           14.0
sexor          250.0
femmor        1847.0
dtype: float64
Kassa (typ=21): -4062.0
[('13', 28969.0), ('16', 28200.0), ('20', 22674.0), ('1', 15146.0), ('14', 5918.5), ('17', 5918.5), ('8', 2671.0), ('11', 2369.5), ('18', 622.0), ('12', 93.0), ('15', -798.0), ('5', -2697.0), ('21', -4062.0), ('3', -8790.5), ('4', -10133.0), ('2', -12006.0), ('6', -14303.0), ('19', -14619.0), ('9', -20244.0), ('10', -21599.0), ('7', -39002.0)]


In [125]:
spel_stats['kassa'] = spel_stats.utdelning-spel_stats.insats
spel_stats.groupby('typ')[['sjuor','sexor','femmor','insats','kassa']].sum().sort_values(by='sjuor', ascending=False)
spel_stats.loc[spel_stats.typ != '7'].sort_values(by='utdelning', ascending=False).head(60)

,typ,datum,insats,utdelning,sjuor,sexor,femmor,kassa
1273,8,2020-10-31,288.0,25775,1,12,58,25487.0
3289,20,2020-10-31,240.0,25735,1,12,56,25495.0
2953,18,2020-10-31,288.0,25468,1,11,50,25180.0
2281,14,2020-10-31,288.0,25468,1,11,50,25180.0
2785,17,2020-10-31,288.0,25468,1,11,50,25180.0
97,1,2020-10-31,288.0,25468,1,11,50,25180.0
2449,15,2020-10-31,288.0,25468,1,11,50,25180.0
2617,16,2020-10-31,256.0,25448,1,11,49,25192.0
2113,13,2020-10-31,256.0,25448,1,11,49,25192.0
1883,12,2019-12-31,324.0,19431,1,11,51,19107.0


In [126]:
stats_typer.columns

Index(['typ', 'ant_datum', 'insats', 'ant_hästar', 'proba', 'kelly', 'motst_diff', 'motst_info',
       'favoriter', 'only_clear', 'streck'],
      dtype='object')

In [127]:
for typ in stats_typer.typ.unique():
    # print(spel_stats.loc[spel_stats.typ == typ][['typ','kassa']])
    stats_typer.loc[stats_typer.typ == typ,'kassa'] = spel_stats[spel_stats.typ==typ].kassa.max()
stats_typer.kassa = stats_typer.kassa.astype('int')

stats_typer.sort_values(by='kassa', ascending=False)
stats_typer.groupby('only_clear')[['kassa']].mean()

,kassa
only_clear,
False,19897.75
True,21643.80
